In [1]:
import os
import glob
import shutil
import re
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
src_folder = r'/awlab/projects/2015_08_Senescence/plates/CH_171219_Vh31_Goldilocks_plate_2017017086_ki67'
img_des_folder = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/plate_2017017086_ki67/images'
mask_des_folder = r'/awlab/users/chsu/WorkSpace/tensorflow/segmentation/data/plate_2017017086_ki67/masks'

file_prefix = 'CH_171219_Vh31_Goldilocks_plate_2017017086_ki67'

# Collect all images

In [3]:
def get_file_list(root_path, pattern='*.png'):
    # Method 1: os.walk
#     img_list = []
#     for root, dirs, files in os.walk(plate_path_root):
#         for f in files:
#             if '.png' in f:
#                 img_list.append(os.path.join(root,f))

    # Method 2: glob
    img_list = [f for f in glob.glob(os.path.join(root_path,'**',pattern), recursive=True)]
    
    return sorted(img_list)

In [4]:
img_list = get_file_list(src_folder, pattern='*-[23].png')

### Copy to destination folder

In [ ]:
if not os.path.isdir(img_des_folder):
    os.mkdir(img_des_folder)

N = len(img_list)
for i, f in enumerate(img_list):
    print('{}/{}'.format(i+1, N), end='\r')
    new_path = os.path.join(img_des_folder, os.path.basename(f))
    shutil.copyfile(f, new_path)

# Get mask from .mat files

In [10]:
mat_files = get_file_list(src_folder, pattern='segmentation-*.mat')
print(len(mat_files))
mat_files[:3]

54


['/awlab/projects/2015_08_Senescence/plates/CH_171219_Vh31_Goldilocks_plate_2017017086_ki67/segmentation/segmentation-2-10.mat',
 '/awlab/projects/2015_08_Senescence/plates/CH_171219_Vh31_Goldilocks_plate_2017017086_ki67/segmentation/segmentation-2-2.mat',
 '/awlab/projects/2015_08_Senescence/plates/CH_171219_Vh31_Goldilocks_plate_2017017086_ki67/segmentation/segmentation-2-3.mat']

In [11]:
def get_well_name(file_name, pattern='segmentation-(?P<row>\d+)-(?P<col>\d+)'):
    m = re.match(pattern, os.path.basename(file_name))
    r_idx = int(m.group('row'))
    well = chr(r_idx+64) + m.group('col')
#     well = chr(r_idx+64) + '{:02d}'.format(int(m.group('col')))

    return well

In [7]:
mat = scipy.io.loadmat(mat_files[0], struct_as_record=False, squeeze_me=True)
seg = mat['segmentation']
s = seg[0]

nuc_mask = s.dnamask>0
nuc_mask = np.uint8(nuc_mask*255.)
cell_mask = np.uint8(s.cellmask*255.)

In [9]:
np.unique(cell_mask)

array([  0, 255], dtype=uint8)

### Save masks to the destination folder

In [12]:
if not os.path.isdir(mask_des_folder):
    os.mkdir(mask_des_folder)
    
for i, f in enumerate(mat_files):
    print('{}/{}'.format(i+1, len(mat_files)), end='\r')
    
    # 1. load the mat file
    mat = scipy.io.loadmat(f, struct_as_record=False, squeeze_me=True)  
    well = get_well_name(f) # get well name
    
    # 2. Get the segmentation struct
    seg = mat['segmentation']

    # 3. Loop through each frame
    for j, s in enumerate(seg):
        nuc_mask = s.dnamask>0
        nuc_mask = np.uint8(nuc_mask*255.)
        cell_mask = np.uint8(s.cellmask*255.)
        
        # 4. Form new file name
        frame_str = '{:04d}'.format(j)
        new_prefix = '_'.join([file_prefix, well, frame_str])
        
        fname_nuc = os.path.join(mask_des_folder, new_prefix + '_nucleus.png')
        fname_cell = os.path.join(mask_des_folder, new_prefix + '_cell.png')
        
        # 5. Save to the mask folder
        cv2.imwrite(fname_nuc, nuc_mask)
        cv2.imwrite(fname_cell, cell_mask)